Loading packages

In [ ]:
import subprocess
import sys

sys.setrecursionlimit(1500)

# Importing the packages
import argparse
import time
import torch
import dgl
import dgl.nn as dglnn
import torch.nn as nn
import torch.nn.functional as F
from dgl import AddSelfLoop
from dgl.data import CiteseerGraphDataset, CoraGraphDataset, PubmedGraphDataset
import numpy as np
import sklearn
from sklearn.model_selection import train_test_split
import matplotlib.pyplot as plt


Extracting graph first as networkx object and then as .graphml file

In [ ]:
import networkx as nx
from neo4j import GraphDatabase
import dgl
import numpy as np

# Neo4j database connection details
uri = "bolt://localhost:7687"
username = "neo4j"
password = "ndTmsTkmT333!"

driver = GraphDatabase.driver(uri, auth=(username, password))

# Query to retrieve nodes and relationships
query = '''
MATCH (n:Gene)-[r:CHROM_INT]->(m:Gene)
RETURN n, r, m, n.difcase, m.difcase, n.ATAC, m.ATAC, n.ATACTR, m.ATACTR, n.Dif_ATAC, m.Dif_ATAC, n.H3K27Ac, m.H3K27Ac, n.H3K27AcTR, m.H3K27AcTR, n.Dif_H3K27Ac, m.Dif_H3K27Ac, n.H3K4Me1, m.H3K4Me1, n.H3K4Me1TR, m.H3K4Me1TR, n.Dif_H3K4Me1, m.Dif_H3K4Me1, n.H3K4Me3, m.H3K4Me3, n.H3K4Me3TR, m.H3K4Me3TR, n.Dif_H3K4Me3, m.Dif_H3K4Me3, n.ISGverse, m.ISGverse, n.HGNCnum, m.HGNCnum;
'''

def process_attribute(value):
    if value in [None, 'NA']:
        return 0
    try:
        return float(value)
    except ValueError:
        return 0

with driver.session() as session:
    result = session.run(query)
    data = result.data()

nx_graph = nx.Graph()

for row in data:
    # Extract the nodes and relationships from the query result
    node1 = row['n']
    rel = row['r']
    node2 = row['m']
    
    # Access the properties with a default value of 0 if not present
    node1_id = node1.get('Ensemblid', 'unknown1')
    node2_id = node2.get('Ensemblid', 'unknown2')

    # Extract attributes, replacing 'NA' with 0
    attributes_n = [
        process_attribute(node1.get('ATAC', 0)), process_attribute(node1.get('ATACTR', 0)), 
        process_attribute(node1.get('Dif_ATAC', 0)), process_attribute(node1.get('H3K27Ac', 0)),
        process_attribute(node1.get('H3K27AcTR', 0)), process_attribute(node1.get('Dif_H3K27Ac', 0)),
        process_attribute(node1.get('H3K4Me1', 0)), process_attribute(node1.get('H3K4Me1TR', 0)), 
        process_attribute(node1.get('Dif_H3K4Me1', 0)), process_attribute(node1.get('H3K4Me3', 0)),
        process_attribute(node1.get('H3K4Me3TR', 0)), process_attribute(node1.get('Dif_H3K4Me3', 0))
    ]

    attributes_m = [
        process_attribute(node2.get('ATAC', 0)), process_attribute(node2.get('ATACTR', 0)), 
        process_attribute(node2.get('Dif_ATAC', 0)), process_attribute(node2.get('H3K27Ac', 0)), 
        process_attribute(node2.get('H3K27AcTR', 0)), process_attribute(node2.get('Dif_H3K27Ac', 0)), 
        process_attribute(node2.get('H3K4Me1', 0)), process_attribute(node2.get('H3K4Me1TR', 0)), 
        process_attribute(node2.get('Dif_H3K4Me1', 0)), process_attribute(node2.get('H3K4Me3', 0)),
        process_attribute(node2.get('H3K4Me3TR', 0)), process_attribute(node2.get('Dif_H3K4Me3', 0))
    ]

    # Add nodes to the graph
    nx_graph.add_node(node1_id, difcase=process_attribute(row.get('n.difcase', 0)), attributes=attributes_n)
    nx_graph.add_node(node2_id, difcase=process_attribute(row.get('m.difcase', 0)), attributes=attributes_m)
    
    # Add edges to the graph
    nx_graph.add_edge(node1_id, node2_id, relationship=type(rel).__name__)

my_dgl_graph = dgl.from_networkx(nx_graph, node_attrs=['difcase', 'attributes'])

dgl.save_graphs('~/70006/graph_MCF7_epiaug6.dgl', [my_dgl_graph])
print(my_dgl_graph)

In [ ]:
import torch
import torchdata
print("torch version:", torch.__version__)
print("torchdata version:", torchdata.__version__)

In [ ]:
#graphs, _=dgl.load_graphs("/rds/general/user/kmt23/home/jobs/epiaug4new.dgl")
graphs, _=dgl.load_graphs("/rds/general/user/kmt23/home/jobs/epiaug6new.dgl")
my_dgl_graph = graphs[0]
print("done")

Setting the features, labels

In [ ]:
labels = my_dgl_graph.ndata['difcase']
features = my_dgl_graph.ndata['attributes']
num_nodes = my_dgl_graph.num_nodes()
num_edges = my_dgl_graph.num_edges()

Setting the training and testing masks

In [ ]:
# Create random masks
# create train, validation and test masks - these define which nodes will be used for training, validation or testing
train_mask = torch.zeros(num_nodes, dtype=bool)
val_mask = torch.zeros(num_nodes, dtype=bool)
test_mask = torch.zeros(num_nodes, dtype=bool)
# First split: train+val and test
train_val_indices, test_indices = train_test_split(torch.arange(num_nodes), test_size=0.2, random_state=42)
# Second split: train and val
train_indices, val_indices = train_test_split(train_val_indices, test_size=0.25, random_state=42)  # 0.25 * 0.8 = 0.2
# Set the mask values to True for the corresponding indices
train_mask[train_indices] = True
val_mask[val_indices] = True
test_mask[test_indices] = True
#masks = torch.from_numpy(train_mask), torch.from_numpy(val_mask), torch.from_numpy(test_mask)




#train_mask = torch.zeros(num_nodes, dtype=torch.bool).bernoulli(0.6)
#val_mask = torch.zeros(num_nodes, dtype=torch.bool).bernoulli(0.2) # make these not overlap
#test_mask = ~(train_mask | val_mask)
    
    # Add masks to the graph
my_dgl_graph.ndata['train_mask'] = train_mask
my_dgl_graph.ndata['val_mask'] = val_mask
my_dgl_graph.ndata['test_mask'] = test_mask
masks = (train_mask, val_mask, test_mask)

Training and testing the GAT - this was done on the HPC with one GPU

In [ ]:
import argparse
import time
import sys
import matplotlib.pyplot as plt
import torch
import torch.nn as nn
import torch.nn.functional as F
import dgl
import dgl.nn as dglnn
from dgl import AddSelfLoop
import numpy as np
from copy import deepcopy


class GAT(nn.Module):
    def __init__(self, in_size, hid_size, out_size, heads):
        super(GAT, self).__init__()
        self.gat_layers = nn.ModuleList()
        # Two-layer GAT
        self.gat_layers.append(
            dglnn.GATConv(
                in_size, hid_size, heads[0], feat_drop=0.05, attn_drop=0.05, activation=F.elu
            )
        )
        self.gat_layers.append(
            dglnn.GATConv(
                hid_size * heads[0], out_size, heads[1], feat_drop=0.05, attn_drop=0.05, activation=None
            )
        )

    def forward(self, g, inputs):
        h = inputs
        for i, layer in enumerate(self.gat_layers):
            h = layer(g, h)
            if i == len(self.gat_layers) - 1:  # last layer
                h = h.mean(1)
            else:  # other layer(s)
                h = h.flatten(1)
        return h


def evaluate(g, features, labels, mask, model):
    model.eval()
    with torch.no_grad():
        logits = model(g, features)
        logits = logits[mask]
        labels = labels[mask]
        _, indices = torch.max(logits, dim=1)
        correct = torch.sum(indices == labels)
        return correct.item() * 1.0 / len(labels)


def train(graph, features, labels, masks, model, num_epochs, patience, class_weights):
    optimizer = torch.optim.Adam(model.parameters(), lr=0.001)
    loss_fcn = nn.CrossEntropyLoss(weight=class_weights)  # CrossEntropyLoss with class weights
    
    train_losses = []  # List to store training losses
    val_losses = []  # List to store validation losses
    best_val_loss = float('inf')
    best_epoch = -1
    best_model_state = None
    patience_counter = 0

    for epoch in range(num_epochs):
        t0 = time.time()
        model.train()
        logits = model(graph, features)
        # logits = model.forward(graph, features)
        loss = loss_fcn(logits[masks[0]], labels[masks[0]].long())
        optimizer.zero_grad()
        loss.backward()
        optimizer.step()

        # Validation step
        val_loss = evaluate_loss(graph, features, labels, masks[1], model, loss_fcn)

        # Collect the loss for this epoch
        train_losses.append(loss.item())
        val_losses.append(val_loss)
        
        if val_loss < best_val_loss:
            best_val_loss = val_loss
            best_epoch = epoch
            best_model_state = deepcopy(model.state_dict())
            patience_counter = 0
        else:
            patience_counter += 1

        t1 = time.time()
        print(f'Epoch {epoch+1}/{num_epochs}, Loss: {loss.item():.4f}, Val Loss: {val_loss:.4f}, Time: {t1 - t0:.4f}')

        # Early stopping check
        if patience_counter >= patience:
            print(f'Early stopping at epoch {epoch+1}')
            break

    # Load the best model state
    model.load_state_dict(best_model_state)

    return train_losses, val_losses, best_epoch, best_val_loss


def evaluate_loss(g, features, labels, mask, model, loss_fcn):
    model.eval()
    with torch.no_grad():
        logits = model(g, features)
        logits = logits[mask]
        labels = labels[mask]
        loss = loss_fcn(logits, labels.long())
        return loss.item()


def parse_args():
    if "ipykernel_launcher" in sys.argv[0]:
        sys.argv = [sys.argv[0]]
    parser = argparse.ArgumentParser()
    parser.add_argument("--num_epochs", type=int, default=4000, help="Number of epochs for training.")
    parser.add_argument("--num_gpus", type=int, default=1, help="Number of GPUs used for training and evaluation.")
    parser.add_argument("--patience", type=int, default=100, help="Early stopping patience.")
    return parser.parse_args()


if __name__ == "__main__":
    args = parse_args()
    
    print("Training with custom DGL graph.")
    
    # Masks need to be predefined or generated appropriately
    # my_dgl_graph, features, labels, and masks should be defined appropriately here

    # Check if GPU is available
    if args.num_gpus > 0 and torch.cuda.is_available():
        device = torch.device("cuda")
    else:
        device = torch.device("cpu")
    
    my_dgl_graph = my_dgl_graph.to(device)
    features = features.to(device)
    labels = labels.to(device)

    # Ensure masks are on the same device
    masks = [mask.to(device) for mask in masks]

    # Create GAT model
    in_size = features.shape[1]
    out_size = 3  # Predicting a categorical label with 3 classes
    model = GAT(in_size, 100, out_size, heads=[8, 8]).to(device)

    # Define class weights, with higher weight for the "difcase" category (assuming it's class 1)
    class_weights = torch.tensor([1.0, 1.0, 1.5], device=device)

    print("Training...")
    train_losses, val_losses, best_epoch, best_val_loss = train(my_dgl_graph, features, labels, masks, model, args.num_epochs, args.patience, class_weights)

    print(f'Best Epoch: {best_epoch+1}, Best Validation Loss: {best_val_loss:.4f}')

    print("Testing...")
    acc = evaluate(my_dgl_graph, features, labels, masks[2], model)
    print("Test accuracy {:.4f}".format(acc))

    # Plot and save training and validation losses
    plt.plot(range(len(train_losses)), train_losses, label='Train Loss')
    plt.plot(range(len(val_losses)), val_losses, label='Validation Loss')
    plt.xlabel('Epoch')
    plt.ylabel('Loss')
    plt.title('Training and Validation Losses')
    plt.legend()
    plt.savefig('train_val_lossesAug4.svg')
    plt.show()

    plt.plot(range(len(train_losses[-300:])), train_losses[-300:], label='Train Loss')
    plt.plot(range(len(val_losses[-300:])), val_losses[-300:], label='Validation Loss')
    plt.xlabel('Epoch')
    plt.ylabel('Loss')
    plt.title('Training and Validation Losses (Last 300 Epochs)')
    plt.legend()
    plt.savefig('train_val_losses_last_300Aug4.svg')
    print('Train losses:', np.round(train_losses, 3))
    print('Validation losses:', np.round(val_losses, 3))

Evaluation

In [ ]:
def evaluate(g, features, labels, mask, model):
    model.eval()
    with torch.no_grad():
        logits = model(g, features)
        logits = logits[mask]
        labels = labels[mask]
        _, predicted = torch.max(logits, dim=1)
        correct = torch.sum(predicted == labels)
        accuracy = correct.item() * 1.0 / len(labels)
    return accuracy, logits, labels

In [ ]:
state_dict = model.state_dict()

In [ ]:
torch.save(state_dict, "best_mcf7gat_modelaug4n.pth")

print("Model state dictionary saved successfully.")

In [ ]:
def get_predictions(g, features, mask, model):
    model.eval()
    with torch.no_grad():
        logits = model(g, features)
        logits = logits[mask]
        _, predicted = torch.max(logits, dim=1)
        return predicted.cpu().numpy()

In [ ]:
def plot_predictions(predictions, true_values, title):
    plt.figure(figsize=(10, 6))
    plt.scatter(range(len(true_values)), true_values, alpha=0.5, label='True Values')
    plt.scatter(range(len(predictions)), predictions, alpha=0.5, label='Predictions')
    plt.xlabel('Sample Index')
    plt.ylabel('Class Label')
    plt.title(title)
    plt.legend()
    plt.show()

In [ ]:
print("Testing...")
test_acc, test_logits, test_labels = evaluate(my_dgl_graph, features, labels, masks[2], model)
print("Test accuracy {:.4f}".format(test_acc))

 # Get predictions for test set
predictions = get_predictions(my_dgl_graph, features, masks[2], model)
true_values = test_labels.cpu().numpy()

    # Plot predictions vs true values
plot_predictions(predictions[:], true_values[:], 'Predictions vs. True Values (Test Set)')

In [ ]:
def component_eval(true_values, predictions):
    print("Total number of samples = ")
    print(len(true_values))
    print("Number of true 0: ")
    true_0 = np.where(true_values == 0)[0]
    print(len(true_0))
    print("Correctly identified as 0: ")
    preds_0 = np.where(predictions == 0)[0]
    
    counter = 0
    for i in range(len(preds_0)):
        if preds_0[i] in true_0:
            counter += 1
    print(counter)
    print("True 0, identified as 1: ")
    preds_1 = np.where(predictions == 1)[0]
    counter = 0
    for i in range(len(preds_1)):
        if preds_1[i] in true_0:
            counter += 1
    print(counter)
    print("Number of true 1: ")
    true_1 = np.where(true_values == 1)[0]
    print(len(true_1))
    print("Correctly identified as 1: ")
    preds_1 = np.where(predictions == 1)[0]
    
    counter = 0
    for i in range(len(preds_1)):
        if preds_1[i] in true_1:
            counter += 1
    print(counter)
    
    print("True 1, identified as 0: ")
    preds_0 = np.where(predictions == 0)[0]
    counter = 0
    for i in range(len(preds_0)):
        if preds_0[i] in true_1:
            counter += 1
    print(counter)
    
    
    print("Number of true 2: ")
    true_2 = np.where(true_values == 2)[0]
    print(len(true_2))
    print("Correctly identified as 2: ")
    preds_2 = np.where(predictions == 2)[0]
    
    counter = 0
    for i in range(len(preds_2)):
        if preds_2[i] in true_2:
            counter += 1
    print(counter)
    print("Predicted # 2: ")
    print(len(preds_2))
    
    print("True 2, identified as 0 or 1: ")
    preds_2 = np.where(predictions != 2)[0]
    counter = 0
    for i in range(len(preds_2)):
        if preds_2[i] in true_2:
            counter += 1
    print(counter)
    
    print("True 1, identified as 2: ")
    preds_2 = np.where(predictions == 2)[0]
    counter = 0
    for i in range(len(preds_0)):
        if preds_2[i] in true_1:
            counter += 1
    print(counter)
    
    print("True 0, identified as 2: ")
    preds_2 = np.where(predictions == 2)[0]
    counter = 0
    for i in range(len(preds_2)):
        if preds_2[i] in true_0:
            counter += 1
    print(counter)
    

In [ ]:
### features = # some features
### forward pass of the whole network is ..
## logit = model(graph, features)
output_of_first_gatconv, attention_weights = model.gat_layers[0](my_dgl_graph, features, get_attention=True)
output_of_first_gatconv = output_of_first_gatconv.flatten(1)
output_of_second_gatconv, attention_weights2 = model.gat_layers[1](my_dgl_graph, output_of_first_gatconv, get_attention=True)

This is where I got the important edges! It says important nodes, but it actually is the edges

In [ ]:
def get_important_nodes(attention_weights, top_k=50):
    # Check if attention_weights has the shape [num_nodes, num_heads, 1]
    assert attention_weights.shape[2] == 1, "Expected attention_weights shape to be [num_nodes, num_heads, 1]"

    # Squeeze the last dimension to get shape [num_nodes, num_heads]
    attention_weights = attention_weights.squeeze(dim=2)  # Shape: [num_nodes, num_heads]

    # Sum the attention weights across all heads
    attention_sums = attention_weights.sum(dim=1)  # Shape: [num_nodes]

    # Get the top_k nodes with the highest attention sums
    if attention_sums.numel() > top_k:  # Ensure there are enough nodes
        important_nodes = torch.topk(attention_sums, top_k).indices
    else:
        important_nodes = torch.arange(attention_sums.numel())  # All nodes if less than top_k
    
    return important_nodes.cpu().numpy()

important_nodes = get_important_nodes(attention_weights)
important_nodes2 = get_important_nodes(attention_weights2)
print(f'Most important nodes: {important_nodes}')
print(f'Most important nodes: {important_nodes2}')

Getting predictions and labels

In [ ]:
def get_predictions_and_labels(g, features, mask, model, true_labels):
    model.eval()
    with torch.no_grad():
        logits = model(g, features)
        logits = logits[mask]
        _, predicted = torch.max(logits, dim=1)
        predictions = predicted.cpu().numpy()
        if isinstance(true_labels, torch.Tensor):
            true_labels = true_labels[mask].cpu().numpy()
        else:
            true_labels = true_labels[mask]  # Assuming true_labels is already a NumPy array
        return predictions, true_labels

Code for GAT figure

In [ ]:
import torch
from sklearn.metrics import f1_score, roc_auc_score, roc_curve, auc, confusion_matrix, ConfusionMatrixDisplay
import matplotlib.pyplot as plt
from sklearn.preprocessing import label_binarize

# Assuming the number of classes
n_classes = len(set(true_labels))

# Binarize the true labels
true_labels_binarized = label_binarize(true_labels, classes=range(n_classes))

# Calculate F1 score
f1 = f1_score(true_labels, predictions, average='weighted')

# Calculate AUC for multi-class classification
with torch.no_grad():
    logits = model(my_dgl_graph, features)
    probs = torch.nn.functional.softmax(logits[masks[2]], dim=1)
    positive_probs = probs.cpu().numpy()  # Multi-class probabilities

# Calculate ROC curve and AUC for each class
fpr = dict()
tpr = dict()
roc_auc = dict()

for i in range(n_classes):
    fpr[i], tpr[i], _ = roc_curve(true_labels_binarized[:, i], positive_probs[:, i])
    roc_auc[i] = auc(fpr[i], tpr[i])

# Plotting F1 score and AUC
plt.figure(figsize=(6, 4))
plt.scatter(true_labels, predictions, alpha=0.5, color="#2E6A57")
plt.xlabel("True Labels")
plt.ylabel("Predicted Labels")
plt.title(f"F1 Score: {f1:.2f} | AUC: {sum(roc_auc.values())/n_classes:.2f}")
plt.grid(False)
plt.savefig('F1_AUC_Plot.png')
plt.show()

colors = ['#142C5F', '#7E8737', '#F6A895', '#124E63', '#FFBAD7']

# Plot ROC Curve for each class
plt.figure(figsize=(6, 4))
for i in range(n_classes):
    plt.plot(fpr[i], tpr[i], color=colors[i % len(colors)], lw=2, label=f'Class {i} (AUC = {roc_auc[i]:.2f})')

plt.plot([0, 1], [0, 1], color='navy', lw=2, linestyle='--')
plt.xlim([0.0, 1.0])
plt.ylim([0.0, 1.05])
plt.xlabel('False Positive Rate')
plt.ylabel('True Positive Rate')
plt.title('Receiver Operating Characteristic')
plt.legend(loc="lower right")
plt.grid(False)
plt.savefig('ROC_Curve.pdf')
plt.show()

# Calculate the Confusion Matrix
conf_matrix = confusion_matrix(true_labels, predictions)

# Display the Confusion Matrix
disp = ConfusionMatrixDisplay(confusion_matrix=conf_matrix, display_labels=range(n_classes))
disp.plot(cmap=plt.cm.Blues)
plt.title('Confusion Matrix')
plt.savefig('Confusion_Matrix.pdf')
plt.show()